In [ ]:
# # Only run this when using kaggle
# ! sudo apt-get update

# # Please install the following packages before running this notebook in google colab/kaggle
# ! sudo apt-get install texlive-latex-recommended 
# ! sudo apt install texlive-latex-extra -y
# ! sudo apt install dvipng -y
# ! sudo apt install cm-super -y
# ! pip install pyopencl
# ! git clone https://github.com/williamjones7/RSCAM-Group-Project- temp;

In [ ]:
# import sys
# sys.path.insert(0, 'temp')

In [ ]:
# Load packages
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyopencl as cl
import pyopencl.array as cl_array

import PyNetworkCPU

from matplotlib import rc
from PyNetwork import ActivationFunctions, ErrorFunctions, MetricFunctions

In [ ]:
# Load context and queue
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

gpu_activation = ActivationFunctions(context, queue)

In [ ]:
# Set the text format same as in the report
rc('font',**{'family':'sans-serif','sans-serif':['Computer Modern Roman'],'size':30})
rc('text', usetex=True)

In [ ]:
def timing_activation_function(x, name):
    """
    Returns average time spent on corresponding activation function
    """

    # Set test data
    x_gpu = cl_array.to_device(queue, x)
    x_tensor = torch.tensor(x)
    x_tensor_gpu = x_tensor.cuda()

    # Load activation functions in PyNetwork
    activation_cpu = PyNetworkCPU.get_activation_function(name)
    activation_gpu = gpu_activation.get_activation_function(name)

    # Compute the running time for PyNetwork
    pynetwork_cpu_time = %timeit -o -q -r 10 -n 100 activation_cpu(x)
    pynetwork_gpu_time = %timeit -o -q -r 10 -n 100 activation_gpu(x_gpu)

    # Compute the running time for PyTorch
    if name == 'relu':
        torch_cpu_time = %timeit -o -q -r 10 -n 100 torch.nn.functional.relu(x_tensor)
        torch_gpu_time = %timeit -o -q -r 10 -n 100 torch.nn.functional.relu(x_tensor_gpu)
    elif name == 'softmax':
        torch_cpu_time = %timeit -o -q -r 10 -n 100 torch.nn.functional.softmax(x_tensor, dim=1)
        torch_gpu_time = %timeit -o -q -r 10 -n 100 torch.nn.functional.softmax(x_tensor_gpu, dim=1)
    
    return pynetwork_cpu_time.average, pynetwork_gpu_time.average, torch_cpu_time.average, torch_gpu_time.average

## Timing for ReLU

In [ ]:
# # Test with different size (square matrix)

# Define x
n_rows = np.arange(0, 4096, 128)
timing_relu = np.zeros((len(n_rows), 4))

# Store the result
for i in range(len(n_rows)):
    x = np.random.rand(n_rows[i], n_rows[i]).astype(np.float32)
    timing_relu[i, :] = timing_activation_function(x, 'relu')

# Plot the result
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

ax.set(xlabel = 'number of rows', ylabel = 'log(Average running time per loop)')
ax.set_yscale('log')
ax.plot(n_rows, timing_relu[:, 0], 'k*-')
ax.plot(n_rows, timing_relu[:, 1], 'b*-')
ax.plot(n_rows, timing_relu[:, 2], 'r*-')
ax.plot(n_rows, timing_relu[:, 3], 'g*-')

plt.legend(['PyNetwork (CPU)', 'PyNetwork (GPU)', 'PyTorch (CPU)', 'PyTorch (GPU)'],
           loc='center right', bbox_to_anchor = (0,-0.1,1,1))

plt.savefig('relu_testing_with_square_matrix.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Test with different size (fixed number of columns, different number of rows)

# Define x
timing_relu_ = np.zeros((len(n_rows), 4))

# Store the result
for i in range(len(n_rows)):
    x = np.random.rand(n_rows[i], 4096).astype(np.float32)
    timing_relu_[i, :] = timing_activation_function(x, 'relu')

# Plot the result
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

ax.set(xlabel = 'number of rows', ylabel = 'log(Average running time per loop)')
ax.set_yscale('log')

ax.plot(n_rows, timing_relu_[:, 0], 'k*-')
ax.plot(n_rows, timing_relu_[:, 1], 'b*-')
ax.plot(n_rows, timing_relu_[:, 2], 'r*-')
ax.plot(n_rows, timing_relu_[:, 3], 'g*-')

plt.legend(['PyNetwork (CPU)', 'PyNetwork (GPU)', 'PyTorch (CPU)', 'PyTorch (GPU)'],
           loc='center right', bbox_to_anchor = (0,-0.1,1,1))

plt.savefig('relu_testing_with_regular_matrix.pdf', bbox_inches='tight')
plt.show()